In [3]:
import os

In [4]:
%pwd

'c:\\Users\\admin\\OneDrive\\Dokumen\\Portofolio\\Data Sceince (Python)\\End-to-end\\research'

In [5]:
os.chdir('../')

In [6]:
%pwd

'c:\\Users\\admin\\OneDrive\\Dokumen\\Portofolio\\Data Sceince (Python)\\End-to-end'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_test_path: Path
    model_path: Path
    metric_file: Path
    all_params: dict
    target_col: str

In [8]:
from end_to_end_project.constants import *
from end_to_end_project.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config['root_dir']])

        model_eval_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_test_path=config.data_test_path,
            model_path=config.model_path,
            metric_file=config.metric_file,
            all_params=params,
            target_col=schema.name
        )

        return model_eval_config

In [ ]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
from dagshub import init
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from end_to_end_project.utils.common import save_json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        init(repo_owner="rajwaAth", repo_name='MLOps-Project', mlflow=True)

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.data_test_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop(columns=[self.config.target_col])
        y_test = test_data[self.config.target_col]

        pred = model.predict(X_test)
        rmse, mae, r2 = self.eval_metrics(y_test, pred)

        scores = {
            "rmse": rmse,
            "mae": mae,
            "r2": r2
        }
        save_json(path=Path(self.config.metric_file), data=scores)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            for key, value in scores.items():
                mlflow.log_metric(key, value)
            mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")





In [11]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(config=model_eval_config)
    model_eval.log_into_mlflow()
except Exception as e:
    raise e 

[2025-08-02 23:15:07,858: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-08-02 23:15:07,866: INFO: common: YAML file: params.yaml loaded successfully]
[2025-08-02 23:15:07,871: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-08-02 23:15:07,875: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-08-02 23:15:08,715: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/rajwaAth/MLOps-Project "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "rajwaAth/MLOps-Project"

[2025-08-02 23:15:08,740: INFO: helpers: Initialized MLflow to track repo "rajwaAth/MLOps-Project"]


Repository rajwaAth/MLOps-Project initialized!

[2025-08-02 23:15:08,744: INFO: helpers: Repository rajwaAth/MLOps-Project initialized!]
[2025-08-02 23:15:08,849: INFO: common: JSON file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\admin\anaconda3\envs\n2nProject\lib\site-packages\_distutils_hack\__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\admin\anaconda3\envs\n2nProject\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
